In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers

In [ ]:
mpl.rc('font', size=14)

dosave = True
savedir = "./plots_1_4_24/preselction_plots/"

import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
MASS = 300

In [ ]:
f = "/icarus/data/users/gputnam/DMCP2023G/mc-F/F2-Higgs_M%i_nom_trk.df" % MASS

In [ ]:
DIST_CUT = 10

In [ ]:
df = pd.read_hdf(f, key="trk")

In [ ]:
mcdf = pd.read_hdf(f, key="mcnu")

In [ ]:
mcdf

In [ ]:
df

In [ ]:
slc_is_reco = SlcInFV(df.slc.vertex) & (df.slc.tmatch.idx == 0)

In [ ]:
two_particles = (df.pfp.dist_to_vertex < DIST_CUT).groupby(level=[0,1,2]).sum() >= 2

In [ ]:
two_contained_particles = ((df.pfp.dist_to_vertex < DIST_CUT) & TrkInFV(df.pfp.trk.end)).groupby(level=[0,1,2]).sum() >= 2

In [ ]:
muid = (df.pfp.dist_to_vertex < DIST_CUT) & TrkInFV(df.pfp.trk.end) & (df.pfp.trk.len > 10) &\
    (df.pfp.trk.chi2pid.I2.chi2_proton > 80) & (df.pfp.trk.chi2pid.I2.chi2_muon < 30)

two_muids = muid.groupby(level=[0,1,2]).sum() >= 2

In [ ]:
two_tracks = ((df.pfp.dist_to_vertex < DIST_CUT) & (df.pfp.trackScore > 0.5)).groupby(level=[0,1,2]).sum() >= 2

In [ ]:
good_reco = (dmagdf(df.pfp.trk.start, df.pfp.trk.truth.p.start) < DIST_CUT) &\
    (dmagdf(df.pfp.trk.end, df.pfp.trk.truth.p.end) < DIST_CUT)

In [ ]:
truthmatch = (df.slc.tmatch.idx == 0) &\
    (df.pfp.trk.truth.p.interaction_id == 0) &\
    (df.pfp.trk.truth.p.start_process == 0)

truthmatch_pos = truthmatch & (df.pfp.trk.truth.p.pdg == 13)
truthmatch_neg = truthmatch & (df.pfp.trk.truth.p.pdg == -13)

good_truthmatch = truthmatch_pos.groupby(level=[0,1,2]).any() & truthmatch_neg.groupby(level=[0,1,2]).any()

In [ ]:
good_recomatch = good_reco & good_truthmatch

In [ ]:
cuts = [
    slc_is_reco,
    two_contained_particles,
    two_muids,
    two_tracks,
]

cutnames = [
    "Slice Reconstructed",
    "Two PFParticles < %.0f cm to Vertex" % DIST_CUT,
    "Two Muons < %.0f cm to Vertex" % DIST_CUT,
    "Two Tracks < %.0f cm to Vertex" % DIST_CUT,
]

In [ ]:
bins = np.linspace(0, 5, 11)
when_all = SlcInFV(mcdf.position) & (mcdf.npi == 0) & (mcdf.npi0 == 0) &\
    TrkInFV(mcdf.p0.end) & TrkInFV(mcdf.p1.end) 

when_cut = when_all
for c, name in zip(cuts, cutnames):
    when_cut = when_cut & (c).groupby(level=[0, 1]).any()

    D, _ = np.histogram(mcdf[when_all].E, bins=bins)
    N, _ = np.histogram(mcdf[when_cut].E, bins=bins)  

    centers = (bins[1:] + bins[:-1])/2.

    plt.plot(centers, N/D, label=name)
    
    
plt.legend()


In [ ]:
cuts = [
    good_recomatch,
    two_contained_particles,
    two_muids,
]

cutnames = [
    "Two Tracks < %.0fcm\nto Vertex" % DIST_CUT,
    "Both Contained",
    "Both Muon ID'd",
]

In [ ]:
bins = np.linspace(0, 5, 11)
when_all = SlcInFV(mcdf.position) & (mcdf.npi == 0) & (mcdf.npi0 == 0) &\
    TrkInFV(mcdf.p0.end) & TrkInFV(mcdf.p1.end)

when_cut = when_all
for c, name in zip(cuts, cutnames):
    when_cut = when_cut & (c).groupby(level=[0, 1]).any()

    D, _ = np.histogram(mcdf[when_all].E, bins=bins)
    N, _ = np.histogram(mcdf[when_cut].E, bins=bins)  

    centers = (bins[1:] + bins[:-1])/2.

    plt.plot(centers, N/D, label=name)
    
plt.legend(loc="lower left")

plt.xlabel("Scalar Energy [GeV]")
plt.ylabel("Selection Fraction")
plt.title("Scalar Portal, $M_S = %i$ MeV" % MASS)

if dosave: plt.savefig(savedir + "scalar_good_reco_M%i_eff.pdf" % MASS)

In [ ]:
lencuts = np.linspace(2, 30, 15)

when_all = SlcInFV(mcdf.position) & (mcdf.npi == 0) & (mcdf.npi0 == 0) &\
    TrkInFV(mcdf.p0.end) & TrkInFV(mcdf.p1.end)

effs = []

for l in lencuts:
    thismuid = (df.pfp.dist_to_vertex < l) & TrkInFV(df.pfp.trk.end) & (df.pfp.trk.len > 10) &\
        (df.pfp.trk.chi2pid.I2.chi2_proton > 80) & (df.pfp.trk.chi2pid.I2.chi2_muon < 30)

    two_thismuids = thismuid.groupby(level=[0,1,2]).sum() >= 2
    
    when_cut = (good_recomatch & two_thismuids).groupby(level=[0, 1]).any()

    eff = when_cut.sum() / when_all.sum()
    
    effs.append(eff)

In [ ]:
plt.plot(lencuts, np.array(effs)*100)
plt.xlabel("Track Distance to Vertex Cut [cm]")
plt.ylabel("Muon ID Efficiency [%]")
plt.title("Scalar Portal, $M_S = %i$ MeV" % MASS)

plt.axvline(DIST_CUT, color="red", linestyle="--")

if dosave: plt.savefig(savedir + "vtxdist_cut_eff_M%i_eff.pdf" % MASS)

In [ ]:
interesting_mcdf = when_all & two_particles.groupby(level=[0,1]).any() & ~two_tracks.groupby(level=[0,1]).any()

In [ ]:
mcdf[interesting_mcdf].position

In [ ]:
good_scalar = (mcdf.npi == 0) & (mcdf.npi0 == 0) &\
    TrkInFV(mcdf.p0.end) & TrkInFV(mcdf.p1.end)

In [ ]:
interesting_slcdf = df.good_scalar & good_reco & ~two_muids & (df.slc.tmatch.idx == 0) & (df.slc.truth.E > 0.5) 

In [ ]:
df["true_contained"] = TrkInFV(df.pfp.trk.truth.p.end)
df["reco_contained"] = TrkInFV(df.pfp.trk.end)

In [ ]:
cols = [
    ("slc", "truth", "E", "", "", ""),
    ("dist_to_vertex", "", "", "", "", ""),
    ("pfp", "trk", "len", "", "", ""),
    ("pfp", "trackScore", "", "", "", ""),
    ("pfp", "trk", "chi2pid", "I2", "chi2_proton", ""),
    ("pfp", "trk", "chi2pid", "I2", "chi2_muon", ""),
#     ("pfp", "trk", "end", "x", "", ""),
#     ("pfp", "trk", "end", "y", "", ""),
#     ("pfp", "trk", "end", "z", "", ""),
#     ("pfp", "trk", "truth", "p", "end", "x"),
#     ("pfp", "trk", "truth", "p", "end", "y"),
#     ("pfp", "trk", "truth", "p", "end", "z"),
    ("true_contained", "", "", "", "", ""),
    ("reco_contained", "", "", "", "", ""),
    ("pfp", "trk", "truth", "p", "pdg", ""),
    ("pfp", "trk", "truth", "p", "length", ""),
    ("pfp", "trk", "truth", "p", "G4ID", ""),
    
]

In [ ]:
df.loc[interesting_slcdf, cols][:50]

In [ ]:
muid.xs((20,15,2))